# RAGStack and Astra vector db

This notebook demonstrates a RAG pattern using RAGStack and the AstraDB vector database.

The pattern is:

1. Construct information base
2. Basic retrieval
3. Generation with augmented context
4. Advanced retrieval and generation
5. Evaluate quality


## Setup
RAGStack includes all the libraries you need for the RAG pattern, including the vector database, embeddings pipeline, and retrieval.

In [ ]:
!pip3 install ragstack-ai datasets

Import the necessary dependencies:

In [ ]:
import getpass
from datasets import load_dataset
from langchain.vectorstores.astradb import AstraDB 
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

If you don't have a vector database, create one at https://astra.datastax.com/.

The Astra application token must have Database Administrator permissions (e.g. `AstraCS:WSnyFUhRxsrg…`​).

The Astra API endpoint is available in the Astra Portal (e.g. `https://<ASTRA_DB_ID>-<ASTRA_DB_REGION>.apps.astra.datastax.com`).

Create an OpenAI key at https://platform.openai.com/ (e.g. `sk-xxxx`).

You must have an existing collection in Astra (e.g. `my-collection`).

Enter your environment variables:

In [ ]:
astra_token = getpass.getpass("Astra token:")
astra_endpoint = getpass.getpass("Astra db endpoint:")
openai_key = getpass.getpass("OpenAI Key:")
collection = input("Collection name:")

## RAG workflow

With your environment set up, you're ready to create a RAG workflow.

### Construct information base

Declare the embeddings model define its required parameters.
Create your Astra vector database and configure its parameters.

In [ ]:
embedding = OpenAIEmbeddings(openai_api_key=openai_key)
vstore = AstraDB(
        collection_name=collection,
        embedding=embedding,
        token=astra_token,
        api_endpoint=astra_endpoint
    )
print("Astra configured")

Load a small dataset of quotes with the Python dataset module.

In [ ]:
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

Process metadata and convert:

In [ ]:
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Add a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

Compute embeddings:

In [ ]:
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")

Confirm your vector store is populated:

In [ ]:
print(vstore.astra_db.collection(collection).find())

### Basic retrieval



Retrieve context from your vector database, and pass it to OpenAI with a prompt question.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

retriever = vstore.as_retriever(search_kwargs={'k': 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI(openai_api_key=openai_key)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what subject are philosophers most concerned with?")
